In [3]:
import os
import cv2
import torch
from torch.utils.data import Dataset

class ShapeDataset(Dataset):
    """
    複数の手書き風図形（円、三角形、四角形）を含む画像と、
    それに対応するラベル情報（クラス、位置、面積）を読み込む PyTorch Dataset クラス。
    """

    def __init__(self, image_dir, label_dir, file_list):
        """
        コンストラクタ

        Args:
            image_dir (str): 画像フォルダへのパス（例："images"）
            label_dir (str): ラベルファイルフォルダへのパス（例："labels"）
            file_list (List[str]): 対象の画像ファイル名（拡張子なし）のリスト（例: ['img_0001', 'img_0002']）
        """
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.file_list = file_list

    def __len__(self):
        """
        データセットのサイズ（サンプル数）を返す

        Returns:
            int: サンプル数（画像数）
        """
        return len(self.file_list)

    def __getitem__(self, idx):
        """
        指定されたインデックスの画像とラベルを返す

        Args:
            idx (int): インデックス

        Returns:
            img (Tensor): 正規化されたグレースケール画像 [1, H, W]
            targets (Tensor): ラベル情報 [num_shapes, 6]
                              各行は [class_id, cx, cy, w, h, area]
        """
        # ファイル名のベース（拡張子なし）を取得
        base_name = self.file_list[idx]

        # 画像とラベルファイルのパスを構築
        img_path = os.path.join(self.image_dir, base_name + ".png")
        label_path = os.path.join(self.label_dir, base_name + ".txt")

        # 画像をグレースケールで読み込み（H, W）、[0, 255] → [0.0, 1.0] に変換
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE).astype("float32") / 255.0

        # [H, W] → [1, H, W] に次元追加（PyTorchの入力に合わせる）
        img = torch.from_numpy(img).unsqueeze(0)

        # ラベル（複数図形）を読み込んで Tensor に変換
        targets = []
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])  # クラス（0=円, 1=三角形, 2=四角形）
                cx, cy, w, h, area = map(float, parts[1:])  # 正規化された中心座標・サイズ・面積
                targets.append([class_id, cx, cy, w, h, area])

        # [num_shapes, 6] の Tensor に変換
        targets = torch.tensor(targets, dtype=torch.float32)

        return img, targets

In [5]:
import os
from torch.utils.data import DataLoader

# --- 設定 ---
image_dir = "images"   # 画像フォルダ
label_dir = "labels"   # ラベルフォルダ

# --- 分割済みファイルリストを読み込む ---
with open("train.txt", "r") as f:
    train_list = [line.strip() for line in f.readlines()]

with open("valid.txt", "r") as f:
    valid_list = [line.strip() for line in f.readlines()]

# --- Datasetのインスタンス作成 ---
train_dataset = ShapeDataset(image_dir, label_dir, train_list)
valid_dataset = ShapeDataset(image_dir, label_dir, valid_list)

# --- DataLoaderの作成 ---
train_loader = DataLoader(
    train_dataset,
    batch_size=32,          # バッチサイズは適宜変更
    shuffle=True,           # 学習時はシャッフル
    drop_last=True          # 最後のバッチを捨てる（サイズが揃わない場合）
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=32,
    shuffle=False,          # 検証はシャッフル不要
    drop_last=False
)

print(f"学習用データ: {len(train_dataset)}枚")
print(f"検証用データ: {len(valid_dataset)}枚")

FileNotFoundError: [Errno 2] No such file or directory: 'train.txt'